## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,76.95,73,74,7.11,PF,2020-12-15 03:46:01
1,1,Buin,-33.73,-70.75,68.00,42,31,6.93,CL,2020-12-15 03:46:03
2,2,Busselton,-33.65,115.33,72.00,35,0,1.99,AU,2020-12-15 03:46:03
3,3,Dikson,73.51,80.55,-19.79,88,74,9.26,RU,2020-12-15 03:42:08
4,4,Rehoboth,-23.32,17.08,57.65,35,9,3.49,NaN,2020-12-15 03:46:03


In [3]:
city_data_df.describe()

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
count,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000,535.000000
mean,267.000000,18.698280,20.114636,50.147290,76.682243,54.000000,8.422561
std,154.585467,33.740853,90.415801,30.488171,17.291507,38.988474,6.467507
min,0.000000,-54.800000,-179.170000,-40.810000,10.000000,0.000000,0.020000
25%,133.500000,-9.735000,-57.950000,30.200000,69.000000,11.000000,3.760000
50%,267.000000,21.850000,24.670000,59.000000,81.000000,67.000000,6.710000
75%,400.500000,47.635000,102.200000,74.325000,89.000000,90.500000,11.410000
max,534.000000,78.220000,179.320000,100.400000,100.000000,100.000000,42.320000


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
minimum = 65 
maximum = 80
cloundiness = 50
wind_speed = 8

In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"]>minimum)|(city_data_df["Max Temp"]<maximum)|(city_data_df["Cloudiness"]<cloundiness)|(city_data_df["Wind Speed"]<wind_speed)]

In [6]:
# 4a. Determine if there are any empty rows.
clean_df.count()


Unnamed: 0    535
City          535
Lat           535
Lng           535
Max Temp      535
Humidity      535
Cloudiness    535
Wind Speed    535
Country       530
Date          535
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna(how='any')

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Rikitea,-23.12,-134.97,76.95,73,74,7.11,PF,2020-12-15 03:46:01
1,1,Buin,-33.73,-70.75,68.00,42,31,6.93,CL,2020-12-15 03:46:03
2,2,Busselton,-33.65,115.33,72.00,35,0,1.99,AU,2020-12-15 03:46:03
3,3,Dikson,73.51,80.55,-19.79,88,74,9.26,RU,2020-12-15 03:42:08
5,5,New Norfolk,-42.78,147.06,77.00,68,40,26.40,AU,2020-12-15 03:46:04
...,...,...,...,...,...,...,...,...,...,...
530,530,Anapskaya,44.89,37.38,41.00,75,44,8.95,RU,2020-12-15 03:47:12
531,531,Taksimo,56.34,114.88,-24.20,77,84,1.83,RU,2020-12-15 03:47:12
532,532,Marawi,8.00,124.29,82.99,81,24,5.01,PH,2020-12-15 03:47:12
533,533,Luba,3.46,8.55,77.00,100,20,4.70,GQ,2020-12-15 03:47:12


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,76.95,-23.12,-134.97,
1,Buin,CL,68.00,-33.73,-70.75,
2,Busselton,AU,72.00,-33.65,115.33,
3,Dikson,RU,-19.79,73.51,80.55,
4,Rehoboth,NaN,57.65,-23.32,17.08,
5,New Norfolk,AU,77.00,-42.78,147.06,
6,Bubaque,GW,79.29,11.28,-15.83,
7,Cabo San Lucas,MX,66.00,22.89,-109.91,
8,Nanortalik,GL,32.95,60.14,-45.24,
9,Victoria,HK,61.00,22.29,114.16,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,76.95,-23.12,-134.97,Pension Maro'i
1,Buin,CL,68.00,-33.73,-70.75,Paihuen
2,Busselton,AU,72.00,-33.65,115.33,Observatory Guest House
3,Dikson,RU,-19.79,73.51,80.55,
4,Rehoboth,NaN,57.65,-23.32,17.08,Noble Luminous Accommodation
...,...,...,...,...,...,...
530,Anapskaya,RU,41.00,44.89,37.38,Continent Hotel
531,Taksimo,RU,-24.20,56.34,114.88,"Gostinitsa ""Viktoriya"""
532,Marawi,PH,82.99,8.00,124.29,Derogongan Residence
533,Luba,GQ,77.00,3.46,8.55,


In [12]:
# 7. Drop the rows where there is no Hotel Name.

nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,76.95,-23.12,-134.97,Pension Maro'i
1,Buin,CL,68.00,-33.73,-70.75,Paihuen
2,Busselton,AU,72.00,-33.65,115.33,Observatory Guest House
4,Rehoboth,NaN,57.65,-23.32,17.08,Noble Luminous Accommodation
5,New Norfolk,AU,77.00,-42.78,147.06,The Shingles Riverside Cottages
...,...,...,...,...,...,...
527,Kurduvadi,IN,74.59,18.08,75.43,Trimurti Hotel and Lodging
528,Tahe,CN,-4.38,52.33,124.73,塔河宾馆
530,Anapskaya,RU,41.00,44.89,37.38,Continent Hotel
531,Taksimo,RU,-24.20,56.34,114.88,"Gostinitsa ""Viktoriya"""


In [16]:
# 8a. Create the output File (CSV)
hotel_df.to_csv("WeatherPy_vacation.csv")


In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
